In [1]:
!pip install -q streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.7 MB/s eta 0:00:00:00:0100:01


In [9]:
%%writefile main_app.py
# --- Core Libraries ---
import streamlit as st
from PIL import Image
import numpy as np
import pandas as pd
import cv2 # OpenCV for image manipulation
import tensorflow as tf
import os

# --- App Configuration ---
st.set_page_config(
    page_title="Chest X-Ray Analysis",
    page_icon="🫁",
    layout="wide",
    initial_sidebar_state="expanded",
)

# --- Custom CSS for UI Enhancement ---
st.markdown("""
<style>
    /* Main app background */
    .stApp {
        background-color: #0E1117;
    }
    /* Card-like containers for sections */
    .st-emotion-cache-1r4qj8v {
        background-color: #161B22;
        border: 1px solid #30363D;
        border-radius: 10px;
        padding: 25px;
        box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    }
    /* Main title styling */
    h1 {
        color: #FFFFFF;
        text-align: center;
    }
    /* Subheader styling */
    h2 {
        color: #58A6FF;
        border-bottom: 2px solid #30363D;
        padding-bottom: 10px;
    }
    /* Styling for the metric label */
    .st-emotion-cache-1g8m52x {
        color: #8B949E;
    }
    /* Analyze button styling */
    .stButton > button {
        background-color: #238636;
        color: white;
        border-radius: 8px;
        border: none;
        padding: 10px 20px;
        width: 100%;
        font-size: 16px;
    }
    .stButton > button:hover {
        background-color: #2EA043;
    }
    .stButton > button:disabled {
        background-color: #30363D;
        color: #8B949E;
    }
    /* File uploader styling */
    .st-emotion-cache-1fttcpj {
        background-color: #0D1117;
        border: 1px dashed #30363D;
    }
</style>
""", unsafe_allow_html=True)


# --- Model & Label Definitions ---
# This MUST match the number of outputs from your model's final layer.
FINAL_14_LABELS = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
    'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule',
    'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
]
IMG_SIZE = 224

# --- Custom Loss Function Definition ---
def get_weighted_loss(weights):
    weights = tf.constant(weights, dtype=tf.float32)
    def weighted_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        loss_weights = (weights[:, 1] * y_true) + (weights[:, 0] * (1 - y_true))
        weighted_bce = loss_weights * bce
        return tf.keras.backend.mean(weighted_bce)
    return weighted_loss

# --- Model Loading (with caching) ---
@st.cache_resource
def load_keras_model(model_path):
    if not os.path.exists(model_path):
        st.error(f"Model file not found at {model_path}. Please check the path.")
        st.info("Make sure you have added your Kaggle Dataset containing the model to this notebook.")
        return None
    try:
        # We need a dummy weights array to pass to the loss function loader
        dummy_weights = np.ones((len(FINAL_14_LABELS), 2))
        custom_objects = {'weighted_loss': get_weighted_loss(dummy_weights)}
        model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
        st.success("Model loaded successfully!")
        return model
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None

# --- Real Prediction & Preprocessing Functions ---
def preprocess_image(image: Image.Image):
    img = image.convert('L') # Convert to grayscale
    img_array = np.array(img)
    
    # Use TensorFlow for preprocessing
    img_tf = tf.convert_to_tensor(img_array, dtype=tf.uint8)
    img_tf = tf.expand_dims(img_tf, axis=-1)
    img_tf = tf.io.decode_png(tf.io.encode_png(img_tf), channels=1)
    img_tf = tf.image.convert_image_dtype(img_tf, tf.float32)
    img_tf = tf.image.grayscale_to_rgb(img_tf)
    img_tf = tf.image.resize(img_tf, [IMG_SIZE, IMG_SIZE])
    return tf.expand_dims(img_tf, axis=0)

def predict_with_model(model, processed_img_batch):
    st.info("Running inference...")
    prediction_array = model.predict(processed_img_batch)[0]
    predictions = dict(zip(FINAL_14_LABELS, prediction_array))
    return predictions

# --- Streamlit App Interface ---
st.title("🫁 Chest X-Ray Diagnostic Assistant")
st.markdown("<p style='text-align: center; color: #8B949E;'>Upload a chest X-ray and patient information for an AI-powered analysis.</p>", unsafe_allow_html=True)
st.markdown("---")

MODEL_PATH = '/kaggle/input/mobilenet-v2-nih-full-dataset-further-fine-tuning/best_chest_xray_model.keras'
model = load_keras_model(MODEL_PATH)

# --- Input Section ---
with st.container(border=True):
    st.header("👤 Patient Information & Upload")
    patient_name = st.text_input("Patient Name", "John Doe")
    patient_age = st.number_input("Patient Age", min_value=0, max_value=120, value=55, step=1)
    patient_gender = st.selectbox("Gender", ["Male", "Female", "Other"])
    
    uploaded_file = st.file_uploader("Upload X-Ray Image", type=['png', 'jpg', 'jpeg'])
    
    st.markdown("<br>", unsafe_allow_html=True) # Spacer
    analyze_button = st.button("Analyze X-Ray", use_container_width=True, disabled=(model is None or uploaded_file is None))

st.markdown("---")

# --- Results Section ---
if analyze_button:
    with st.container(border=True):
        st.header("📊 Analysis Results")
        image = Image.open(uploaded_file)
        st.image(image, caption=f"Uploaded X-Ray for {patient_name}", use_container_width=True)
        
        with st.spinner("Running analysis... Please wait."):
            # Preprocess image and get predictions
            processed_img_batch = preprocess_image(image)
            predictions = predict_with_model(model, processed_img_batch)
            
        st.success("Analysis Complete!")
        st.subheader("🩺 Model Predictions")
        pred_df = pd.DataFrame(predictions.items(), columns=['Condition', 'Probability']).sort_values(by='Probability', ascending=False).reset_index(drop=True)
        top_prediction_df = pred_df.iloc[0]
        st.metric(label="Most Likely Condition", value=top_prediction_df['Condition'], delta=f"{top_prediction_df['Probability']:.2%}")
        
        with st.expander("View Full Probability Distribution"):
            st.bar_chart(pred_df.set_index('Condition'))
else:
    st.info("Please provide patient info, upload an X-ray, and click 'Analyze' to view the results.")


Overwriting main_app.py


In [10]:
from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok

# Get the authtoken from Kaggle Secrets
try:
    user_secrets = UserSecretsClient()
    NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")
except:
    print("Ngrok authtoken not found. Please add it to this notebook's secrets.")
    NGROK_AUTH_TOKEN = ""

if NGROK_AUTH_TOKEN:
    # Terminate any existing ngrok tunnels
    ngrok.kill()

    # Set the authtoken for ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Set up the public URL.
    public_url = ngrok.connect(8501)
    print(f"Click to open your Streamlit app: {public_url}")

    # Run the Streamlit app
    !streamlit run main_app.py

Click to open your Streamlit app: NgrokTunnel: "https://d8daf4e01d89.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.75.234.110:8501

2025-07-08 15:55:00.519141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751990100.548871     242 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751990100.557746     242 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-08 15:55:05.415860: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call